In [1]:
import pandas as pd
import re
from textblob import TextBlob
from gensim import corpora
from gensim import similarities
import time
import nltk
from textblob.en import sentiment as pattern_sentiment
from textblob.en.sentiments import PatternAnalyzer as pattern_analyzer

unable to import 'smart_open.gcs', disabling that module


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/phanivalasa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/phanivalasa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
pd.set_option('display.max_colwidth', 500)

In [5]:
tweets = pd.read_csv('../data/raw/tweetsCID_cosine_bow.csv', dtype=str)

In [6]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80467 entries, 0 to 80466
Data columns (total 11 columns):
tweet_id                             80433 non-null object
tweet_text                           80403 non-null object
user_id                              80369 non-null object
tweet_time                           80335 non-null object
no_favs                              80335 non-null object
no_retweets                          80335 non-null object
urls                                 80335 non-null object
content_id                           80335 non-null object
tweet_text_url_token                 80301 non-null object
tweet_urls                           80301 non-null object
tweet_text_url_token_preprocessed    80301 non-null object
dtypes: object(11)
memory usage: 6.8+ MB


In [7]:
# tweets[65500:65530]

## Textblob subjectivity

In [8]:
def obj_tuples(input):
    ret_list = []
    ret_string = ''
    for i in range(len(input)):
        if input[i][1] == 0:
            tpl = (" ".join(input[i][0]), input[i][1])
            ret_list.append(tpl)
#         ret_string = ret_string+str(tpl)
        
    return ret_list

In [9]:
def sub_tuples(input):
    ret_list = []
    ret_string = ''
    for i in range(len(input)):
        if input[i][1] != 0:
            tpl = (" ".join(input[i][0]), input[i][1])
            ret_list.append(tpl)
#         ret_string = ret_string+str(tpl)
        
    return ret_list

In [10]:
def process_tweets_textblob(df):    
    column_names = ['tweet_id','content_id','tweet_text_url_token','polarity_score', 'sentiment','subjective',
                    'sub_assessments','obj_assessments']
    # polarity_score - is the polarity score
    # sentiment - pos, neg or neu
    # subjective - True/False. True if it's neg or pos.
    # sub_assessments - words contributing to pos/neg

    return_df = pd.DataFrame()
    for idx,row in df.iterrows():
        tweet_id = row['tweet_id']
        content_id = row['content_id']
        tweet_text_url_token = row['tweet_text_url_token']
#         sub = TextBlob(row['tweet_text_url_token']).sentiment.subjectivity
        polarity_score = TextBlob(str(row['tweet_text_url_token'])).sentiment.polarity
        if (polarity_score>0):
            sentiment = 'pos'
            subjective = True
        elif (polarity_score<0):
            sentiment = 'neg'
            subjective = True
        else:
            sentiment = 'neu'
            subjective = False
        sub_assessments = sub_tuples(pattern_sentiment(row['tweet_text_url_token']).assessments)
        obj_assessments = obj_tuples(pattern_sentiment(row['tweet_text_url_token']).assessments)
        return_df = pd.concat([return_df, pd.DataFrame([[tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,
                                                         subjective, sub_assessments,obj_assessments]])],axis=0)
    
    return_df.columns=column_names
    return_df.index =return_df['tweet_id']
    return return_df

In [11]:
df_tweet = tweets[:]

In [12]:
len(df_tweet)

80467

In [13]:
sentiment_scores = process_tweets_textblob(df_tweet)

### Subjective Tweets

In [297]:
sentiment_scores[sentiment_scores['subjective']==True][['tweet_text_url_token', 'content_id',
                                                        'polarity_score', 'sentiment','subjective',
                                                        'sub_assessments', 'obj_assessments']].head(30)

,tweet_text_url_token,content_id,polarity_score,sentiment,subjective,sub_assessments,obj_assessments
tweet_id,,,,,,,
8.51485E+17,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,2736,0.200000,pos,True,"[(ecological, 0.4)]","[(financial, 0.0)]"
8.55009E+17,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,2644,0.050000,pos,True,"[(clear, 0.10000000000000002)]","[(not actually, -0.0)]"
8.55012E+17,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,2644,-1.000000,neg,True,"[(horrible, -1.0)]",[]
8.61559E+17,How an elephant tranquilizer became the latest deadly drug in the opioid epidemic <URL>,1909,0.150000,pos,True,"[(latest, 0.5), (deadly, -0.2)]",[]
8.61566E+17,"""When it comes to cracking down on opioids, just going after the drug’s supply isn’t enough."" <URL> by @germanrlopez",1909,-0.077778,neg,True,"[(down, -0.15555555555555559)]","[(enough, 0.0)]"
8.6157E+17,"Very real news. If you can (literally) even physically see the amount of this you are taking, it's enough to kill you <URL>",1909,0.086667,pos,True,"[(very real, 0.26)]","[(physically, 0.0), (enough, 0.0)]"
8.61574E+17,Crack down on opiods and heroin and people start using horse tranquilizers. You have to provide treatment to help. <URL>,1909,-0.155556,neg,True,"[(down, -0.15555555555555559)]",[]
8.61575E+17,"Another article that can't keep prescription vs illegal opioid use straight. Too bad, @voxdotcom-could've been good <URL>",1909,-0.075000,neg,True,"[(illegal, -0.5), (straight, 0.2), (bad, -0.6999999999999998), (good, 0.7)]",[]
8.61586E+17,Veterinary tranquilizer one of latest deadly drug in the opioid epidemic #veterinary @whcucdavis <URL> via @voxdotcom,1909,0.150000,pos,True,"[(latest, 0.5), (deadly, -0.2)]",[]


### Objective Tweets

In [24]:
sentiment_scores[sentiment_scores['subjective']==False][['tweet_text_url_token','content_id',
                                                        'polarity_score', 'sentiment','subjective',
                                                        'sub_assessments','obj_assessments']]

,tweet_text_url_token,content_id,polarity_score,sentiment,subjective,sub_assessments,obj_assessments
tweet_id,,,,,,,
8.5501E+17,Hmmmmmm <URL>,2644,0.0,neu,False,[],[]
8.55015E+17,Hello <URL>,2644,0.0,neu,False,[],[]
8.5502E+17,SOMEBODY hasn't read about prohibition yet. <URL>,2644,0.0,neu,False,[],[]
8.61565E+17,<URL> via @,1909,0.0,neu,False,[],[]
8.61571E+17,<URL> #fentanyl #opioidepidemic $BICX,1909,0.0,neu,False,[],[]
...,...,...,...,...,...,...,...
9.30509E+17,Comments from Ryder and our automated vehicle technology partner @embarktrucks were featured in this @nytimes story about #AutonomousVehicles and how soon we could expect to regularly see them on the highway. <URL>,3351,0.0,neu,False,[],"[(regularly, 0.0)]"
9.30527E+17,😨\n<URL> <URL>,3351,0.0,neu,False,[],[]
9.3053E+17,I've got to figure out a way to make money on this transition. <URL>,3351,0.0,neu,False,[],[]


In [16]:
# sentiment_scores[(sentiment_scores['sub']<0.1) & (sentiment_scores['sub_assessments'] != '')][['tweet_text_url_token','sub','sub_assessments']]

## VADER Sentiment Analysis

In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize, RegexpTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/phanivalasa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/phanivalasa/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [18]:
analyzer = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(text):
    score = analyzer.polarity_scores(text)['compound']
    return score
#     print(text)
#     print(score)

In [19]:
def get_word_sentiment(text):
    
    tokenized_text = nltk.word_tokenize(text)
    
    sub_word_list=[]
    neu_word_list=[]
    
    for word in tokenized_text:
        score = analyzer.polarity_scores(word)['compound']
        if score > 0.05:
            tup = (word, score)
            sub_word_list.append(tup)
        elif score < -0.05:
            tup = (word, score)
            sub_word_list.append(tup)
        else:
            neu_word_list.append(word)
            
#     print('Subjective:',sub_word_list)        
#     print('Neutral:',neu_word_list)    
    return sub_word_list


In [20]:
# text_pos = '#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>'
# sentiment_analyzer_scores(text_pos)

In [21]:
# get_word_sentiment(text_pos)

In [25]:
def process_tweets_vader(df):    
    column_names = ['tweet_id','content_id','tweet_text_url_token','polarity_score', 'sentiment','subjective',
                    'sub_assessments','obj_assessments']
    # polarity_score - is the polarity score
    # sentiment - pos, neg or neu
    # subjective - True/False. True if it's neg or pos.
    # sub_assessments - words contributing to pos/neg

    return_df = pd.DataFrame()
    for idx,row in df.iterrows():
        tweet_id = row['tweet_id']
        content_id = row['content_id']
        tweet_text_url_token = row['tweet_text_url_token']
#         sub = TextBlob(row['tweet_text_url_token']).sentiment.subjectivity
        polarity_score = sentiment_analyzer_scores(str(row['tweet_text_url_token']))
        if (polarity_score>0.05):
            sentiment = 'pos'
            subjective = True
        elif (polarity_score<-0.05):
            sentiment = 'neg'
            subjective = True
        else:
            sentiment = 'neu'
            subjective = False
        sub_assessments = get_word_sentiment(str(row['tweet_text_url_token']))
        obj_assessments = ''
        return_df = pd.concat([return_df, pd.DataFrame([[tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,
                                                         subjective, sub_assessments,obj_assessments]])],axis=0)
    
    return_df.columns=column_names
    return_df.index =return_df['tweet_id']
    return return_df

In [26]:
sentiment_scores_vader = process_tweets_vader(df_tweet)

In [61]:
sentiment_scores_vader[sentiment_scores_vader['subjective']==True][['tweet_text_url_token','content_id',
                                                        'polarity_score', 'sentiment','subjective',
                                                        'sub_assessments', 'obj_assessments']]

,tweet_text_url_token,content_id,polarity_score,sentiment,subjective,sub_assessments,obj_assessments
tweet_id,,,,,,,
8.51485E+17,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,2736,-0.6249,neg,True,"[(disaster, -0.6249)]",
8.55009E+17,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,2644,0.7641,pos,True,"[(clear, 0.3818), (Tougher, 0.1779), (security, 0.34), (stop, -0.296)]",
8.55012E+17,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,2644,-0.8856,neg,True,"[(distract, -0.296), (horrible, -0.5423), (racism, -0.6249)]",
8.6157E+17,"Very real news. If you can (literally) even physically see the amount of this you are taking, it's enough to kill you <URL>",1909,-0.6908,neg,True,"[(kill, -0.6908)]",
8.61574E+17,Crack down on opiods and heroin and people start using horse tranquilizers. You have to provide treatment to help. <URL>,1909,-0.2263,neg,True,"[(heroin, -0.4939), (tranquilizers, -0.1027), (help, 0.4019)]",
...,...,...,...,...,...,...,...
9.30497E+17,Harvey Mudd alumni are impacting #selfdriving auto innovations. Read about '01 engineering alum &amp; @PelotonTech CEO Josh Switkes' work to make trucking safer. <URL>,3351,0.4215,pos,True,"[(safer, 0.4215)]",
9.3051E+17,But not a great headline for truck drivers <URL>,3351,-0.6642,neg,True,"[(great, 0.6249)]",
9.30527E+17,😨\n<URL> <URL>,3351,-0.4939,neg,True,"[(😨, -0.4939)]",


## Stanford CoreNLP

In [28]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

In [48]:
def get_sentiment_score(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 5000,
                       })
#     print(text)
#     print('Sentiment:', res['sentences'][0]['sentiment'])
#     print('Sentiment score:', res['sentences'][0]['sentimentValue'])
#     print('Sentiment distribution (0-v. negative, 5-v. positive:', res['sentences'][0]['sentimentDistribution'])
    ret = res['sentences'][0]['sentimentValue']

    if ret.isdecimal():
        ret = int(ret)
    else:
        print('Non int value - ', ret)
        ret = 2
    return ret

In [49]:
def get_sentiment_score_stanford(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 5000,
                       })
    
    count = 0
    score = 0
    
    try:         
        for s in res["sentences"]:
            count += 1
            sc = int(s["sentimentValue"])
            score = score + sc
    #         print("{}: '{}': {} (Sentiment Value) {} (Sentiment)".format(
    #             s["index"],
    #             " ".join([t["word"] for t in s["tokens"]]),
    #             s["sentimentValue"], s["sentiment"]))

        ret = score/count
    except:
        print('failed text is - ', text)
        ret = 2

    return ret

In [50]:
def process_tweets_stanford_nlp(df):    
    column_names = ['tweet_id','content_id','tweet_text_url_token','polarity_score', 'sentiment','subjective',
                    'sub_assessments','obj_assessments']
    # polarity_score - is the polarity score
    # sentiment - pos, neg or neu
    # subjective - True/False. True if it's neg or pos.
    # sub_assessments - words contributing to pos/neg

    return_df = pd.DataFrame()
    for idx,row in df.iterrows():
        tweet_id = row['tweet_id']
        content_id = row['content_id']
        tweet_text_url_token = row['tweet_text_url_token']
#         sub = TextBlob(row['tweet_text_url_token']).sentiment.subjectivity
        polarity_score = get_sentiment_score_stanford(str(row['tweet_text_url_token']))
        if (polarity_score<2):
            sentiment = 'neg'
            subjective = True
        elif (polarity_score>2):
            sentiment = 'pos'
            subjective = True
        else:
            sentiment = 'neu'
            subjective = False
        sub_assessments = ''
        obj_assessments = ''
        return_df = pd.concat([return_df, pd.DataFrame([[tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,
                                                         subjective, sub_assessments,obj_assessments]])],axis=0)
    
    return_df.columns=column_names
    return_df.index =return_df['tweet_id']
    return return_df

In [51]:
sentiment_scores_stanford = process_tweets_stanford_nlp(df_tweet)

In [293]:
sentiment_scores_stanford[sentiment_scores_stanford['subjective']==True][['tweet_text_url_token', 'content_id',
                                                        'polarity_score', 'sentiment','subjective',
                                                        'sub_assessments', 'obj_assessments']]

,tweet_text_url_token,content_id,polarity_score,sentiment,subjective,sub_assessments,obj_assessments
tweet_id,,,,,,,
8.51485E+17,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,2736,1.000000,neg,True,,
8.55009E+17,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,2644,1.000000,neg,True,,
8.55012E+17,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,2644,1.800000,neg,True,,
8.5502E+17,SOMEBODY hasn't read about prohibition yet. <URL>,2644,1.500000,neg,True,,
8.61566E+17,"""When it comes to cracking down on opioids, just going after the drug’s supply isn’t enough."" <URL> by @germanrlopez",1909,1.500000,neg,True,,
...,...,...,...,...,...,...,...
9.30554E+17,"In the past year, self-driving &amp; other trucking technologies have seen more than $1 billion in investments <URL>",3351,1.000000,neg,True,,
9.30562E+17,@theitskeptic Yet more cherry-picking... <URL>,3351,1.000000,neg,True,,
9.30571E+17,"As readers of <URL> already know thanks to our coverage of UPS, FedEx investments in driverless trucks, drones, and other job-killers... <URL>",3351,1.000000,neg,True,,


## Analysis

In [68]:
sentiment_scores_vader[sentiment_scores_vader['subjective']==False][['content_id']].count()

content_id    25343
dtype: int64

#### Read topics

In [72]:
url = 'https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/processed/article_topics_6.csv'

In [77]:
raw_topics = pd.read_csv(url)

In [78]:
topics = raw_topics[['content_id','topic']]
topics = topics.set_index(['content_id'])
topics.head()

,topic
content_id,
60,Policy
61,Policy
66,Economy
80,Policy
82,Public Health


#### Read News Source

In [84]:
raw_articles = pd.read_csv("../data/raw/knight_data_articles.csv")

In [85]:
articles = raw_articles[['content_id','content_source_desc']]
articles = articles.set_index(['content_id'])
articles.head()

,content_source_desc
content_id,
60,Fox News
61,Fox News
66,Fox News
80,The New York Times
82,Vox


#### Read Mean ratings

In [87]:
raw_ratings = pd.read_csv("../data/raw/knight_data_ratings.csv")

In [91]:
raw_ratings_blind = raw_ratings[raw_ratings['blind']==1][['content_id','rating_scale_response']]
blind_ratings = raw_ratings_blind.groupby(['content_id']).mean()
# blind_ratings['content_id1'] = blind_ratings.index
blind_ratings.head()

,rating_scale_response
content_id,
60,5.00
61,4.50
66,3.50
80,3.75
82,3.50


#### Articles Source and Mean Ratings

In [198]:
articles_ratings = articles.join(blind_ratings)
articles_ratings_topics = articles_ratings.join(topics)

In [130]:
articles_ratings['content_id'] = articles_ratings.index
articles_ratings['content_id_idx'] = articles_ratings.index
articles_ratings = articles_ratings.set_index('content_id_idx')

In [131]:
articles_ratings.head()

,content_source_desc,rating_scale_response,content_id
content_id_idx,,,
60,Fox News,5.00,60
61,Fox News,4.50,61
66,Fox News,3.50,66
80,The New York Times,3.75,80
82,Vox,3.50,82


In [201]:
articles_ratings_topics['content_id'] = articles_ratings_topics.index
articles_ratings_topics['content_id_idx'] = articles_ratings_topics.index
articles_ratings_topics = articles_ratings_topics.set_index('content_id_idx')

In [202]:
articles_ratings_topics.head()

,content_source_desc,rating_scale_response,topic,content_id
content_id_idx,,,,
60,Fox News,5.00,Policy,60
61,Fox News,4.50,Policy,61
66,Fox News,3.50,Economy,66
80,The New York Times,3.75,Policy,80
82,Vox,3.50,Public Health,82


In [203]:
source_rating = articles_ratings_topics[['content_source_desc','rating_scale_response']].groupby(['content_source_desc']).mean()
source_rating.head()

,rating_scale_response
content_source_desc,
100 Percent Fed Up,2.242219
Associated Press News,3.524466
Breitbart,2.525192
Fox News,3.253773
Media Matters,2.559345


In [204]:
topic_rating = articles_ratings_topics[['topic','rating_scale_response']].groupby(['topic']).mean()
topic_rating.head()

,rating_scale_response
topic,
Economy,3.209807
Policy,3.179498
Politics,3.080381
Public Health,3.595236
Scandal,3.038835


## VADER Analysis
#### Merge Additional details in Sentiment score dataframe

In [205]:
articles_ratings_topics['content_id'] = articles_ratings_topics['content_id'].astype(str)

In [206]:
df_vader = pd.merge(sentiment_scores_vader,articles_ratings_topics, on='content_id', how='inner')

In [207]:
df_vader.head()

,tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,subjective,sub_assessments,obj_assessments,content_source_desc,rating_scale_response,topic
0,8.51485E+17,2736,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,-0.6249,neg,True,"[(disaster, -0.6249)]",,Vox,3.615385,Public Health
1,8.55009E+17,2644,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,0.7641,pos,True,"[(clear, 0.3818), (Tougher, 0.1779), (security, 0.34), (stop, -0.296)]",,Vox,3.421053,Public Health
2,8.5501E+17,2644,Hmmmmmm <URL>,0.0000,neu,False,[],,Vox,3.421053,Public Health
3,8.55012E+17,2644,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,-0.8856,neg,True,"[(distract, -0.296), (horrible, -0.5423), (racism, -0.6249)]",,Vox,3.421053,Public Health
4,8.55015E+17,2644,Hello <URL>,0.0000,neu,False,[],,Vox,3.421053,Public Health


In [226]:
df_vader.to_csv('../data/raw/vader_tweet_subjectivity_analysis.csv')

In [225]:
# df_vader[['subjective','content_source_desc']].groupby('content_source_desc').count()

In [227]:
def create_source_df(df_sentiment):
    
    df_analysis_sentiment = pd.DataFrame(columns=['content_source_desc', 'deduped_tweets', 'objective_tweets', 'subjective_tweets', 
                                            'positive_tweets','negative_tweets','neutral_tweets','trust_mean'])

    for index, row in source_rating.iterrows():

        df_analysis_sentiment = df_analysis_sentiment.append({'content_source_desc':index, 
          'deduped_tweets':df_sentiment[(df_sentiment['content_source_desc']==index) ][['subjective']].count()[0], 
          'objective_tweets':df_sentiment[(df_sentiment['content_source_desc']==index) & (df_sentiment['subjective']==False) ][['subjective']].count()[0], 
          'subjective_tweets':df_sentiment[(df_sentiment['content_source_desc']==index) & (df_sentiment['subjective']==True) ][['subjective']].count()[0], 
          'positive_tweets':df_sentiment[(df_sentiment['content_source_desc']==index) & (df_sentiment['sentiment']=='pos') ][['subjective']].count()[0], 
          'negative_tweets':df_sentiment[(df_sentiment['content_source_desc']==index) & (df_sentiment['sentiment']=='neg') ][['subjective']].count()[0], 
          'neutral_tweets':df_sentiment[(df_sentiment['content_source_desc']==index) & (df_sentiment['sentiment']=='neu') ][['subjective']].count()[0], 
          'trust_mean':row['rating_scale_response']}, ignore_index = True)


    df_analysis_sentiment['subjective_tweets_proportion'] = df_analysis_sentiment['subjective_tweets']/(df_analysis_sentiment['subjective_tweets']+df_analysis_vader['objective_tweets'])
    df_analysis_sentiment['pos/neg Ratio'] = df_analysis_sentiment['positive_tweets']/df_analysis_sentiment['negative_tweets']
    
    return df_analysis_sentiment

In [228]:
df_analysis_vader = create_source_df(df_vader)

In [230]:
df_analysis_vader[['content_source_desc','deduped_tweets','subjective_tweets_proportion','pos/neg Ratio','trust_mean']]

,content_source_desc,deduped_tweets,subjective_tweets_proportion,pos/neg Ratio,trust_mean
0,100 Percent Fed Up,1871,0.686799,0.649551,2.242219
1,Associated Press News,417,0.645084,0.977941,3.524466
2,Breitbart,9735,0.675295,0.736856,2.525192
3,Fox News,11711,0.694817,0.717029,3.253773
4,Media Matters,1234,0.628849,0.637131,2.559345
5,The New York Times,43466,0.688653,0.945217,3.527990
6,Vox,11867,0.675655,0.829758,3.403589


In [232]:
def create_topic_df(df_sentiment):
    
    df_analysis_sentiment = pd.DataFrame(columns=['topic', 'deduped_tweets', 'objective_tweets', 'subjective_tweets', 
                                            'positive_tweets','negative_tweets','neutral_tweets','trust_mean'])

    for index, row in topic_rating.iterrows():

        df_analysis_sentiment = df_analysis_sentiment.append({'topic':index, 
          'deduped_tweets':df_sentiment[(df_sentiment['topic']==index) ][['subjective']].count()[0], 
          'objective_tweets':df_sentiment[(df_sentiment['topic']==index) & (df_sentiment['subjective']==False) ][['subjective']].count()[0], 
          'subjective_tweets':df_sentiment[(df_sentiment['topic']==index) & (df_sentiment['subjective']==True) ][['subjective']].count()[0], 
          'positive_tweets':df_sentiment[(df_sentiment['topic']==index) & (df_sentiment['sentiment']=='pos') ][['subjective']].count()[0], 
          'negative_tweets':df_sentiment[(df_sentiment['topic']==index) & (df_sentiment['sentiment']=='neg') ][['subjective']].count()[0], 
          'neutral_tweets':df_sentiment[(df_sentiment['topic']==index) & (df_sentiment['sentiment']=='neu') ][['subjective']].count()[0], 
          'trust_mean':row['rating_scale_response']}, ignore_index = True)


    df_analysis_sentiment['subjective_tweets_proportion'] = df_analysis_sentiment['subjective_tweets']/(df_analysis_sentiment['subjective_tweets']+df_analysis_vader['objective_tweets'])
    df_analysis_sentiment['pos/neg Ratio'] = df_analysis_sentiment['positive_tweets']/df_analysis_sentiment['negative_tweets']
    
    return df_analysis_sentiment

In [233]:
df_analysis_vader_topic = create_topic_df(df_vader)
df_analysis_vader_topic[['topic','deduped_tweets', 'subjective_tweets_proportion', 'pos/neg Ratio','trust_mean']]

,topic,deduped_tweets,subjective_tweets_proportion,pos/neg Ratio,trust_mean
0,Economy,1868,0.682385,1.30165,3.209807
1,Policy,16965,0.988143,0.951582,3.179498
2,Politics,24359,0.843313,0.772926,3.080381
3,Public Health,15688,0.742841,0.887386,3.595236
4,Scandal,18966,0.96485,0.746839,3.038835
5,Technology,2455,0.0991813,1.69439,3.267849


## TextBlob Analysis
#### Merge Additional details in Sentiment score dataframe

In [234]:
# articles_ratings_topics['content_id'] = articles_ratings_topics['content_id'].astype(str)

In [242]:
df_textblob = pd.merge(sentiment_scores,articles_ratings_topics, on='content_id', how='inner')

In [243]:
df_textblob.head()

,tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,subjective,sub_assessments,obj_assessments,content_source_desc,rating_scale_response,topic
0,8.51485E+17,2736,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,0.20,pos,True,"[(ecological, 0.4)]","[(financial, 0.0)]",Vox,3.615385,Public Health
1,8.55009E+17,2644,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,0.05,pos,True,"[(clear, 0.10000000000000002)]","[(not actually, -0.0)]",Vox,3.421053,Public Health
2,8.5501E+17,2644,Hmmmmmm <URL>,0.00,neu,False,[],[],Vox,3.421053,Public Health
3,8.55012E+17,2644,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,-1.00,neg,True,"[(horrible, -1.0)]",[],Vox,3.421053,Public Health
4,8.55015E+17,2644,Hello <URL>,0.00,neu,False,[],[],Vox,3.421053,Public Health


In [244]:
df_textblob.to_csv('../data/raw/textblob_tweet_subjectivity_analysis.csv')

In [245]:
df_analysis_textblob = create_source_df(df_textblob)

In [246]:
df_analysis_textblob[['content_source_desc','deduped_tweets','subjective_tweets_proportion','pos/neg Ratio','trust_mean']]

,content_source_desc,deduped_tweets,subjective_tweets_proportion,pos/neg Ratio,trust_mean
0,100 Percent Fed Up,1871,0.645278,1.21622,2.242219
1,Associated Press News,417,0.580737,1.69737,3.524466
2,Breitbart,9735,0.615871,1.17791,2.525192
3,Fox News,11711,0.63979,1.10897,3.253773
4,Media Matters,1234,0.595763,1.1987,2.559345
5,The New York Times,43466,0.646593,1.60412,3.527990
6,Vox,11867,0.629904,1.58933,3.403589


In [247]:
df_analysis_textblob_topic = create_topic_df(df_textblob)
df_analysis_textblob_topic[['topic','deduped_tweets', 'subjective_tweets_proportion', 'pos/neg Ratio','trust_mean']]

,topic,deduped_tweets,subjective_tweets_proportion,pos/neg Ratio,trust_mean
0,Economy,1868,0.648892,2.48232,3.209807
1,Policy,16965,0.985241,1.53009,3.179498
2,Politics,24359,0.813135,1.26271,3.080381
3,Public Health,15688,0.705091,1.69558,3.595236
4,Scandal,18966,0.956674,1.30996,3.038835
5,Technology,2455,0.0874579,2.27525,3.267849


## Stanford CoreNLP Analysis
#### Merge Additional details in Sentiment score dataframe

In [234]:
# articles_ratings_topics['content_id'] = articles_ratings_topics['content_id'].astype(str)

In [235]:
df_stanford = pd.merge(sentiment_scores_stanford,articles_ratings_topics, on='content_id', how='inner')

In [236]:
df_stanford.head()

,tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,subjective,sub_assessments,obj_assessments,content_source_desc,rating_scale_response,topic
0,8.51485E+17,2736,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,1.0,neg,True,,,Vox,3.615385,Public Health
1,8.55009E+17,2644,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,1.0,neg,True,,,Vox,3.421053,Public Health
2,8.5501E+17,2644,Hmmmmmm <URL>,2.0,neu,False,,,Vox,3.421053,Public Health
3,8.55012E+17,2644,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,1.8,neg,True,,,Vox,3.421053,Public Health
4,8.55015E+17,2644,Hello <URL>,2.0,neu,False,,,Vox,3.421053,Public Health


In [237]:
df_stanford.to_csv('../data/raw/stanford_tweet_subjectivity_analysis.csv')

In [238]:
df_analysis_stanford = create_source_df(df_stanford)

In [239]:
df_analysis_stanford[['content_source_desc','deduped_tweets','subjective_tweets_proportion','pos/neg Ratio','trust_mean']]

,content_source_desc,deduped_tweets,subjective_tweets_proportion,pos/neg Ratio,trust_mean
0,100 Percent Fed Up,1871,0.694951,0.164921,2.242219
1,Associated Press News,417,0.68172,0.205323,3.524466
2,Breitbart,9735,0.687525,0.156853,2.525192
3,Fox News,11711,0.699512,0.156841,3.253773
4,Media Matters,1234,0.650915,0.213068,2.559345
5,The New York Times,43466,0.6994,0.199002,3.527990
6,Vox,11867,0.689997,0.210541,3.403589


In [248]:
df_analysis_stanford_topic = create_topic_df(df_stanford)
df_analysis_stanford_topic[['topic','deduped_tweets', 'subjective_tweets_proportion', 'pos/neg Ratio','trust_mean']]

,topic,deduped_tweets,subjective_tweets_proportion,pos/neg Ratio,trust_mean
0,Economy,1868,0.699179,0.20318,3.209807
1,Policy,16965,0.988375,0.178735,3.179498
2,Politics,24359,0.846486,0.183299,3.080381
3,Public Health,15688,0.760616,0.223575,3.595236
4,Scandal,18966,0.966936,0.164493,3.038835
5,Technology,2455,0.112183,0.280899,3.267849


## VADER, TEXTBLOB, CORE NLP - Results 

In [268]:
df_analysis_source = pd.DataFrame(columns=['content_source_desc', 'deduped_tweets', 'VADER Sub Proportion', 'VADER pos/neg', 
                                        'TEXTBLOB Sub Proportion','TEXTBLOG pos/neg',
                                        'STANFORD Sub Proportion','Stanford pos/neg'])


for index, row in source_rating.iterrows():
#     print(index,df_analysis_vader[df_analysis_vader['content_source_desc']==index].reset_index().at[0,'subjective_tweets_proportion'])

    df_analysis_source = df_analysis_source.append({'content_source_desc':index, 
      'deduped_tweets':df_analysis_vader[df_analysis_vader['content_source_desc']==index].reset_index().at[0,'deduped_tweets'], 
      'VADER Sub Proportion':df_analysis_vader[df_analysis_vader['content_source_desc']==index].reset_index().at[0,'subjective_tweets_proportion'], 
      'VADER pos/neg':df_analysis_vader[df_analysis_vader['content_source_desc']==index].reset_index().at[0,'pos/neg Ratio'], 
      'TEXTBLOB Sub Proportion':df_analysis_textblob[df_analysis_textblob['content_source_desc']==index].reset_index().at[0,'subjective_tweets_proportion'],
      'TEXTBLOG pos/neg':df_analysis_textblob[df_analysis_textblob['content_source_desc']==index].reset_index().at[0,'pos/neg Ratio'], 
      'STANFORD Sub Proportion':df_analysis_stanford[df_analysis_stanford['content_source_desc']==index].reset_index().at[0,'subjective_tweets_proportion'],
      'Stanford pos/neg':df_analysis_stanford[df_analysis_stanford['content_source_desc']==index].reset_index().at[0,'pos/neg Ratio']}, 
                            ignore_index = True)

df_analysis_source

,content_source_desc,deduped_tweets,VADER Sub Proportion,VADER pos/neg,TEXTBLOB Sub Proportion,TEXTBLOG pos/neg,STANFORD Sub Proportion,Stanford pos/neg
0,100 Percent Fed Up,1871,0.686799,0.649551,0.645278,1.216216,0.694951,0.164921
1,Associated Press News,417,0.645084,0.977941,0.580737,1.697368,0.681720,0.205323
2,Breitbart,9735,0.675295,0.736856,0.615871,1.177911,0.687525,0.156853
3,Fox News,11711,0.694817,0.717029,0.639790,1.108970,0.699512,0.156841
4,Media Matters,1234,0.628849,0.637131,0.595763,1.198697,0.650915,0.213068
5,The New York Times,43466,0.688653,0.945217,0.646593,1.604123,0.699400,0.199002
6,Vox,11867,0.675655,0.829758,0.629904,1.589328,0.689997,0.210541


In [269]:
# df_analysis_vader

In [271]:
df_analysis_topic = pd.DataFrame(columns=['topic', 'deduped_tweets', 'VADER Sub Proportion', 'VADER pos/neg', 
                                        'TEXTBLOB Sub Proportion','TEXTBLOG pos/neg',
                                        'STANFORD Sub Proportion','Stanford pos/neg'])


for index, row in topic_rating.iterrows():
#     print(index,df_analysis_vader[df_analysis_vader['topic']==index].reset_index().at[0,'subjective_tweets_proportion'])

    df_analysis_topic = df_analysis_topic.append({'topic':index, 
      'deduped_tweets':df_analysis_vader_topic[df_analysis_vader_topic['topic']==index].reset_index().at[0,'deduped_tweets'], 
      'VADER Sub Proportion':df_analysis_vader_topic[df_analysis_vader_topic['topic']==index].reset_index().at[0,'subjective_tweets_proportion'], 
      'VADER pos/neg':df_analysis_vader_topic[df_analysis_vader_topic['topic']==index].reset_index().at[0,'pos/neg Ratio'], 
      'TEXTBLOB Sub Proportion':df_analysis_textblob_topic[df_analysis_textblob_topic['topic']==index].reset_index().at[0,'subjective_tweets_proportion'],
      'TEXTBLOG pos/neg':df_analysis_textblob_topic[df_analysis_textblob_topic['topic']==index].reset_index().at[0,'pos/neg Ratio'], 
      'STANFORD Sub Proportion':df_analysis_stanford_topic[df_analysis_stanford_topic['topic']==index].reset_index().at[0,'subjective_tweets_proportion'],
      'Stanford pos/neg':df_analysis_stanford_topic[df_analysis_stanford_topic['topic']==index].reset_index().at[0,'pos/neg Ratio']}, 
                            ignore_index = True)


df_analysis_topic

,topic,deduped_tweets,VADER Sub Proportion,VADER pos/neg,TEXTBLOB Sub Proportion,TEXTBLOG pos/neg,STANFORD Sub Proportion,Stanford pos/neg
0,Economy,1868,0.682385,1.301645,0.648892,2.482315,0.699179,0.203180
1,Policy,16965,0.988143,0.951582,0.985241,1.530090,0.988375,0.178735
2,Politics,24359,0.843313,0.772926,0.813135,1.262708,0.846486,0.183299
3,Public Health,15688,0.742841,0.887386,0.705091,1.695584,0.760616,0.223575
4,Scandal,18966,0.964850,0.746839,0.956674,1.309959,0.966936,0.164493
5,Technology,2455,0.099181,1.694394,0.087458,2.275253,0.112183,0.280899


.

### Vader

In [299]:
sentiment_scores_vader[ (sentiment_scores_vader['tweet_id']=='8.51485E+17') | (sentiment_scores_vader['tweet_id']=='8.62834E+17')][['tweet_text_url_token', 'content_id',
                                                        'polarity_score', 'sentiment','subjective',
                                                        'sub_assessments']]

,tweet_text_url_token,content_id,polarity_score,sentiment,subjective,sub_assessments
tweet_id,,,,,,
8.51485E+17,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,2736,-0.6249,neg,True,"[(disaster, -0.6249)]"
8.62834E+17,"""After hundreds and hundreds of studies in thousands of children"" ... &lt;amateurish statement lacking scientific process &amp; direct evidence&gt; <URL>",2795,0.0000,neu,False,"[(gt, 0.2732)]"


.

### Textblob

In [300]:
sentiment_scores[ (sentiment_scores['tweet_id']=='8.51485E+17') | (sentiment_scores['tweet_id']=='8.62834E+17')][['tweet_text_url_token', 'content_id',
                                                        'polarity_score', 'sentiment','subjective',
                                                        'sub_assessments']]

,tweet_text_url_token,content_id,polarity_score,sentiment,subjective,sub_assessments
tweet_id,,,,,,
8.51485E+17,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,2736,0.2,pos,True,"[(ecological, 0.4)]"
8.62834E+17,"""After hundreds and hundreds of studies in thousands of children"" ... &lt;amateurish statement lacking scientific process &amp; direct evidence&gt; <URL>",2795,0.1,pos,True,"[(direct, 0.1)]"


.

### Stanford CoreNLP

In [301]:
sentiment_scores_stanford[ (sentiment_scores_stanford['tweet_id']=='8.51485E+17') | (sentiment_scores_stanford['tweet_id']=='8.62834E+17')][['tweet_text_url_token', 'content_id',
                                                        'polarity_score', 'sentiment','subjective',
                                                        'sub_assessments']]

,tweet_text_url_token,content_id,polarity_score,sentiment,subjective,sub_assessments
tweet_id,,,,,,
8.51485E+17,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,2736,1.0,neg,True,
8.62834E+17,"""After hundreds and hundreds of studies in thousands of children"" ... &lt;amateurish statement lacking scientific process &amp; direct evidence&gt; <URL>",2795,1.0,neg,True,


.

In [125]:
sentiment_scores_vader.head()

,tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,subjective,sub_assessments,obj_assessments
tweet_id,,,,,,,,
8.51485E+17,8.51485E+17,2736,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,-0.6249,neg,True,"[(disaster, -0.6249)]",
8.55009E+17,8.55009E+17,2644,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,0.7641,pos,True,"[(clear, 0.3818), (Tougher, 0.1779), (security, 0.34), (stop, -0.296)]",
8.5501E+17,8.5501E+17,2644,Hmmmmmm <URL>,0.0000,neu,False,[],
8.55012E+17,8.55012E+17,2644,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,-0.8856,neg,True,"[(distract, -0.296), (horrible, -0.5423), (racism, -0.6249)]",
8.55015E+17,8.55015E+17,2644,Hello <URL>,0.0000,neu,False,[],


In [112]:
sentiment_scores_vader

,tweet_id,content_id,tweet_text_url_token,polarity_score,sentiment,subjective,sub_assessments,obj_assessments
tweet_id,,,,,,,,
8.51485E+17,8.51485E+17,2736,#Trump’s border wall would be an ecological and financial disaster for the U.S. <URL>,-0.6249,neg,True,"[(disaster, -0.6249)]",
8.55009E+17,8.55009E+17,2644,The research and experts are clear: Tougher border security measures do not actually stop drug trafficking <URL>,0.7641,pos,True,"[(clear, 0.3818), (Tougher, 0.1779), (security, 0.34), (stop, -0.296)]",
8.5501E+17,8.5501E+17,2644,Hmmmmmm <URL>,0.0000,neu,False,[],
8.55012E+17,8.55012E+17,2644,(psst! It was never about drugs. It's always been about brown people. They holler DRUGS! to distract from their horrible racism) <URL>,-0.8856,neg,True,"[(distract, -0.296), (horrible, -0.5423), (racism, -0.6249)]",
8.55015E+17,8.55015E+17,2644,Hello <URL>,0.0000,neu,False,[],
...,...,...,...,...,...,...,...,...
9.30562E+17,9.30562E+17,3351,@theitskeptic Yet more cherry-picking... <URL>,0.0000,neu,False,[],
9.30566E+17,9.30566E+17,3351,Big increase in investment hastening arrival of driverless trucks – at least in the US #transport #logistics <URL>,0.3182,pos,True,"[(increase, 0.3182)]",
9.30571E+17,9.30571E+17,3351,"As readers of <URL> already know thanks to our coverage of UPS, FedEx investments in driverless trucks, drones, and other job-killers... <URL>",0.4404,pos,True,"[(thanks, 0.4404)]",


In [108]:
sentiment_scores_vader.count()

tweet_id                80433
content_id              80335
tweet_text_url_token    80301
polarity_score          80467
sentiment               80467
subjective              80467
sub_assessments         80467
obj_assessments         80467
dtype: int64

In [115]:
df_vader = sentiment_scores_vader.set_index('content_id').join(articles)

In [116]:
df_vader

,tweet_id,tweet_text_url_token,polarity_score,sentiment,subjective,sub_assessments,obj_assessments,content_source_desc
content_id,,,,,,,,
#CorruptGOP puts thru 4 #Unqualified judges nominated by #Trump. #Alabama gets 1. VoteTheGOPout,#Alabamasenate #RoyMooreChildMolestor https://t.co/uTK2zxyAk6,NaN,0.0,neu,False,[],,NaN
#CorruptGOP will soon transfer him to a useless position for talking about #ClimateChange.,#MoronsAreGoverning https://t.co/Iye7UCcyfR,NaN,0.0,neu,False,[],,NaN
#GNTP Great New Tax Plan <URL> T,This plan is not simple- where are the massive cuts? @realDonaldTrump @GOPSenFinance,NaN,0.0,neu,False,[],,NaN
#PruittEndsEPA,#CorruptGOP #Idiots https://t.co/8kxn9nT6cS,NaN,0.0,neu,False,[],,NaN
#SecuredBorders reposted video attributed to Conservative Tribune,https://t.co/uYeYCUDoCr,NaN,0.0,neu,False,[],,NaN
...,...,...,...,...,...,...,...,...
NaN,#Alabamasenate #RoyMooreChildMolestor <URL>,NaN,0.0,neu,False,[],,NaN
NaN,9.29827E+17,NaN,0.0,neu,False,[],,NaN
NaN,HOW IS THIS EVEN POSSIBLE?! WOW! <URL>,NaN,0.0,neu,False,[],,NaN


In [106]:
df_vader.count()

tweet_id                80433
tweet_text_url_token    80301
polarity_score          80467
sentiment               80467
subjective              80467
sub_assessments         80467
obj_assessments         80467
dtype: int64

In [109]:
df_vader = df_vader.join(articles)

In [110]:
df_vader.head()

,tweet_id,tweet_text_url_token,polarity_score,sentiment,subjective,sub_assessments,obj_assessments,content_source_desc
content_id,,,,,,,,
#CorruptGOP puts thru 4 #Unqualified judges nominated by #Trump. #Alabama gets 1. VoteTheGOPout,#Alabamasenate #RoyMooreChildMolestor https://t.co/uTK2zxyAk6,NaN,0.0,neu,False,[],,NaN
#CorruptGOP will soon transfer him to a useless position for talking about #ClimateChange.,#MoronsAreGoverning https://t.co/Iye7UCcyfR,NaN,0.0,neu,False,[],,NaN
#GNTP Great New Tax Plan <URL> T,This plan is not simple- where are the massive cuts? @realDonaldTrump @GOPSenFinance,NaN,0.0,neu,False,[],,NaN
#PruittEndsEPA,#CorruptGOP #Idiots https://t.co/8kxn9nT6cS,NaN,0.0,neu,False,[],,NaN
#SecuredBorders reposted video attributed to Conservative Tribune,https://t.co/uYeYCUDoCr,NaN,0.0,neu,False,[],,NaN


In [92]:
blind_ratings.count()

rating_scale_response    1627
dtype: int64

In [94]:
articles.count()

content_source_desc    1914
dtype: int64

In [95]:
topics.count()

topic    1914
dtype: int64

In [97]:
articles_ratings = articles.join(blind_ratings)

In [99]:
articles_ratings.count()

content_source_desc      1914
rating_scale_response    1627
dtype: int64

In [101]:
articles_ratings

,content_source_desc,rating_scale_response
content_id,,
60,Fox News,5.00
61,Fox News,4.50
66,Fox News,3.50
80,The New York Times,3.75
82,Vox,3.50
...,...,...
3380,Associated Press News,3.50
3381,Associated Press News,NaN
3382,Associated Press News,4.50


In [ ]:
raw_ratings = pd.read_csv("../data/knight_data_ratings.csv")
raw_ratings.head()